In [1]:
import numpy as np
import keras
from keras.models import Model
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D
import tensorflow.keras.preprocessing.image as prep #데이터 Augmentation, 하면 좋은거임

Using TensorFlow backend.


In [2]:
def VGG16_Brief(classes=2): # classes = 감지할 클래스 수
    img_rows, img_cols = 224, 224
    img_channels = 3    
    img_dim = (img_rows, img_cols, img_channels) #차원..shape 정의..
    
    img_input = Input(shape=img_dim) #튜플.. 변경 x
    x = Conv2D(64,(3,3),padding='same',activation = 'relu')(img_input) #1층
    x = Conv2D(64,(3,3),padding='same',activation = 'relu')(x) #2층
    x = MaxPooling2D((2,2),strides=(2,2))(x) #2층 maxpool

    x = Conv2D(128,(3,3),padding='same',activation = 'relu')(x) #3층
    #x = Conv2D(128,(3,3),padding='same',activation = 'relu')(x) #4층
    x = MaxPooling2D((2,2),strides=(2,2))(x) #4층 maxpool

    x = Conv2D(256,(3,3),padding='same',activation = 'relu')(x) #5층
    #x = Conv2D(256,(3,3),padding='same',activation = 'relu')(x) #6층
    x = Conv2D(256,(3,3),padding='same',activation = 'relu')(x) #7층
    x = MaxPooling2D((2,2),strides=(2,2))(x) #7층 maxpool

    #x = Conv2D(512,(3,3),padding='same',activation = 'relu')(x) #8층
    #x = Conv2D(512,(3,3),padding='same',activation = 'relu')(x) #9층
    #x = Conv2D(512,(3,3),padding='same',activation = 'relu')(x) #10층
    #x = MaxPooling2D((2,2),strides=(2,2))(x) #10층 maxpool

    #x = Conv2D(512,(3,3),padding='same',activation = 'relu')(x) #11층
    #x = Conv2D(512,(3,3),padding='same',activation = 'relu')(x) #12층
    #x = Conv2D(512,(3,3),padding='same',activation = 'relu')(x) #13층
    #x = MaxPooling2D((2,2),strides=(2,2))(x) #13층 maxpool

    x = Flatten()(x)
    x = Dense(256, activation = 'relu')(x) #14층
    x = Dense(256, activation = 'relu')(x) #15층
    x = Dense(classes, activation = 'softmax')(x)
    
    model = Model(inputs=img_input, output = x)
    return model

In [3]:
model = VGG16_Brief(classes = 2)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 256)       2951

C:\Users\notou\Anaconda3\envs\tf_gpu_10\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [4]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0003), 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
cat_dog_data_dir='C:\dataset\data_100'

train_datagen = prep.ImageDataGenerator(
rescale=1./255, #normalization 이미지 픽셀 값이 1/225를 곱해준다. default = 1
shear_range=0.2,#반시계 방향 밀리기
zoom_range=0.2, #줌
horizontal_flip=True #인풋을 무작위로 가로로 뒤집기
)

train_generator = train_datagen.flow_from_directory( #폴더 형태로된 데이터 구조 바로 가저옴
                                                    #이 데이터 불러올 떄 앞에 정의한 파라미터로 전처리
cat_dog_data_dir,
target_size=(224,224), #폴더 내 이미지 사이즈 조정
batch_size=32,  #32장씩 읽어들임
#shuffle=True,
class_mode='categorical' #2D one-hot 인코딩된 라벨 반환
)

Found 2000 images belonging to 2 classes.


In [6]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='accuracy', 
                                            patience=5, #5번동안 accuracy 안늘면 
                                            verbose=1, #업데이트 되면 말해주는 것
                                            factor=0.1, 
                                            min_lr=0.00001)

In [7]:
history = model.fit(train_generator, epochs = 10, callbacks=[learning_rate_reduction])
#generator을 input으로 사용할 떄는 batch_size 이용하지 않음 
#batch_size 32 = 32장씩 읽어드리고 가중치 갱신 
#batch_size 값이 크면 클수록 여러 데이터를 기억하고 있어야 하기에 메모리가 커야 합니다. 그대신 학습 시간이 빨라집니다. 
#batch_size 값이 작으면 학습은 꼼꼼하게 이루어질 수 있지만 학습 시간이 많이 걸립니다.

Epoch 1/10
63/63 [==============================] - 26s 406ms/step - loss: 0.7011 - accuracy: 0.5110
Epoch 2/10
63/63 [==============================] - 25s 392ms/step - loss: 0.6931 - accuracy: 0.5045
Epoch 3/10
63/63 [==============================] - 26s 406ms/step - loss: 0.6941 - accuracy: 0.5035
Epoch 4/10
63/63 [==============================] - 28s 438ms/step - loss: 0.6936 - accuracy: 0.5065
Epoch 5/10
63/63 [==============================] - 26s 418ms/step - loss: 0.6925 - accuracy: 0.5215
Epoch 6/10
63/63 [==============================] - 27s 423ms/step - loss: 0.6787 - accuracy: 0.5720
Epoch 7/10
63/63 [==============================] - 27s 422ms/step - loss: 0.6682 - accuracy: 0.6070
Epoch 8/10
63/63 [==============================] - 25s 399ms/step - loss: 0.6155 - accuracy: 0.6560
Epoch 9/10
63/63 [==============================] - 25s 399ms/step - loss: 0.5863 - accuracy: 0.6910
Epoch 10/10
63/63 [==============================] - 27s 422ms/step - loss: 0.5514 - accura

In [ ]:
#validation set은 여러 모델 중에서 최종 모델을 선정하기 위한 성능 평가에 관여한다 보시면됩니다. 
#따라서 validation set은 training과정에 관여하게 됩니다. 즉, validation set은 training 과정에 관여를 하며, 
#training이 된 여러가지 모델 중 가장 좋은 하나의 모델을 고르기 위한 셋입니다. 
#오버피팅 파악하고 피하기.
#test set은 모든 training 과정이 완료된 후에 최종적으로 모델의 성능을 평가하기 위한 셋입니다. 



In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history["accuracy"], label= 'ACCURACY')
plt.plot(history.history['loss'], label= 'LOSS')
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(loc='lower right')
plt.show()